# Raspberry Pi Cameraとperftoolのデータをサーバに送信する

## 概要

### システム構成

システム構成を次図に示します。`Raspberry Pi`で示された枠内がこのnotebookで構築する対象になります。

![システム構成](img/14-components.png)
<!--
```mermaid
flowchart LR
  subgraph S[server]
    subgraph N["NATS"]
      PTT(["perftool target topic"])
    end
    subgraph K["Apache Kafka"]
      CT(["camera topic"])
      PTR(["perftool result topic"])
    end
  end
  subgraph R["Raspberry Pi"]
    C(["Camera"])
    CC["sensor-picamera\nContainer"]
    PC["perftool\nContainer"]
    C---CC==>CT
    PC-.->PTT
    PC==>PTR
  end
```
-->

sensor-picameraコンテナはRaspberry Piに接続されたカメラの画像を取得して、そのデータをKafkaブローカに送信します。またカメラ撮影の合間に[perftoolコマンド](https://github.com/nii-gakunin-cloud/sinetstream/tree/main/java/sample/perftool)を実行して、その計測結果をKafkaブローカに送信します。

### 前提条件

環境を構築、実行するために必要となる前提条件を示します。

* Raspberry Pi OS (64-bit)
* [Raspberry Pi カメラモジュール](https://www.raspberrypi.com/documentation/accessories/camera.html)
  * USBカメラはサポート対象外になります
* docker, docker compose v2
  * カメラ画像を送信するコンテナを実行するために必要となリます

前提条件に関する簡単なチェックを行います。

構築環境のOSをチェックします。次のセルを実行してエラーにならないことを確認して下さい。

In [ ]:
lsb_release -d | grep -e 'Debian GNU/Linux 12 (bookworm)'

docker, docker composeが利用可能になっていることを確認します。

In [ ]:
docker version

In [ ]:
docker compose version

## パラメータの指定

### 配置場所 

環境を構築するために必要となる資材を配置するディレクトリを次のセルに指定してください。

In [ ]:
# (例)
# target_dir=$HOME/picamera
# target_dir=/srv/picamera

target_dir=

### picamera

カメラデータを送信するコンテナsensor-picameraではRaspberry Piカメラでの撮影に[Picamera2](https://github.com/raspberrypi/picamera2)を利用します。ここではPicamera2に設定するパラメータを指定します。

カメラの画像サイズを指定します。画像の幅と高さを(width)x(height)で指定することができます。また典型的なサイズについては文字列で指定することもできます。

* QVGA
  * 320x240
* VGA
  * 640x480
* HD
  * 1280x720
* FHD, 2K
  * 1920x1080
* 4K
  * 3840x2160

In [ ]:
# (例)
# PICAMERA2_SIZE=320x240
# PICAMERA2_SIZE=VGA
# PICAMERA2_SIZE=640x480
# PICAMERA2_SIZE=FHD

PICAMERA2_SIZE=

> `4K`などの大きなサイズの画像を指定する場合は、追加で最大メッセージサイズなどの設定を変更する必要があります。

画像サイズ以外のPicamera2のパラメータに関してもコンテナの環境変数により設定することができます。指定する環境変数名はPicamera2のコンフィギュレーションパラメータを以下のルールで変換したものになります。

* 全て大文字に変換する
* プレフィックスに `PICAMERA2_` をつける
* パラメータ名をプレフィックスの後に続ける
  * 例えばcolour_spaceに対応する環境変数名は`PICAMERA2_COLOUR_SPACE`になる
* stream-specific configurationパラメータはプレフィックスの後にパラメータ名を_でつなげる
  * 例えばsizeに対応する環境変数名は`PICAMERA2_SIZE`になる
* camera controlsのように子要素があるパラメータは、パラメータ名の後に_でつなげる
* またcontrol名のように大文字、小文字が混在している場合は大文字の箇所を_で区切る
  * 例えばcontrol名AwbModeに対応する環境変数名は`PICAMERA2_CONTROLS_AWB_MODE`になる
  
環境変数の設定例を次の表に示します。

|環境変数名|説明|設定例|
|---|---|---|
|PICAMERA2_TRANSFORM_HFLIP|水平反転|PICAMERA2_TRANSFORM_HFLIP=1|
|PICAMERA2_TRANSFORM_VFLIP|垂直反転|PICAMERA2_TRANSFORM_VFLIP=1|
|PICAMERA2_BUFFER_COUNT|バッファセット数|PICAMERA2_BUFFER_COUNT=2|
|PICAMERA2_CONTROLS_FRAME_RATE|フレームレート|PICAMERA2_CONTROLS_FRAME_RATE=5|
|PICAMERA2_CONTROLS_AF_MODE|オートフォーカス|PICAMERA2_CONTROLS_AF_MODE=Continuous|
|PICAMERA2_CONTROLS_AWB_MODE|ホワイトバランス|PICAMERA2_CONTROLS_AWB_MODE=Tungsten|


画像サイズ以外のpicamera2パラメータを設定する場合は次のセルで指定して下さい。

In [ ]:
# (例)
# PICAMERA2_PARAMS="
# PICAMERA2_TRANSFORM_HFLIP=1
# PICAMERA2_TRANSFORM_VFLIP=1
# "

PICAMERA2_PARAMS="
"

### 撮影間隔

カメラ画像を取得する時間間隔（秒）を環境変数`SCHEDULE`に指定して下さい。

In [ ]:
# (例)
# SCHEDULE=60

SCHEDULE=60

`SCHEDULE`を指定しない場合は可能な限り連続的に画像を送信し続けます。

### 画像送信先
カメラデータの送信先となるKafkaブローカに関するパラメータを指定します。

Kafkaの外部公開ホスト名を指定してください。Raspberry PiからKafkaブローカにアクセスするときは、ここで指定したホスト名（またはIPアドレス）でアクセス出来るように設定する必要があります。

In [ ]:
# (例)
# kafka_host=kafka.example.org
# kafka_host=192.168.10.100

kafka_hostname=

Kafkaブローカのポート番号を指定して下さい。

In [ ]:
# (例)
# kafka_port=9092

kafka_port=9092

カメラデータの送信先となるKafkaのトピック名を指定してください。ここで指定する値は、サーバを構築した際に指定した値と一致したものである必要があります。

In [ ]:
# (例)
# picamera_topic=image-sinetstream-picamera

picamera_topic=image-sinetstream-picamera

### perftool

[perftoolコマンド](https://github.com/nii-gakunin-cloud/sinetstream/tree/main/java/sample/perftool)のオプションに対応する環境変数を指定します。

「[perftool - オプション](https://github.com/nii-gakunin-cloud/sinetstream/tree/main/java/sample/perftool#%E3%82%AA%E3%83%97%E3%82%B7%E3%83%A7%E3%83%B3)」に記述されているオプションのうち、以下に示したものを指定することができます。

* -c, --output-count OUTPUT_COUNT
  * OUTPUT_COUNT回メトリクス情報を計測し出力する。デフォルト:1
* -n, --num-samples NUM_SAMPLES
  * 一回のメトリクス情報を出力するまでにwriterがNUM_SAMPLES個のデータ送信を試行する。デフォルト:300
* -p, --payload-size PAYLOAD_SIZE
  * データ送信試行時にPAYLOAD_SIZEバイトのペイロード長のデータを使用する。デフォルト:1024
* -a, --async-api
  * 指定した場合はデータの送信/受信に非同期APIを使用する。指定しなかった場合は同期APIを使用する
* -t, --num-threads NUM_THREADS
  * NUM_THREADS個のreader/writerのペアを並列実行する。各ペアはOUTPUT_COUNT回メトリクス情報を計測する。デフォルト:1

perftoolコマンドのオプションと環境変数の対応を次表に示します。

|オプション|環境変数|指定例|
|---|---|---|
|--output-count|EXT_ARG_OUTPUT_COUNT|EXT_ARG_OUTPUT_COUNT=1|
|--num-samples|EXT_ARG_NUM_SAMPLES|EXT_ARG_NUM_SAMPLES=300|
|--payload-size|EXT_ARG_PAYLOAD_SIZE|EXT_ARG_PAYLOAD_SIZE=1024|
|--async-api|EXT_ARG_ASYNC_API|EXT_ARG_ASYNC_API=1|
|--num-threads|EXT_ARG_NUM_THREADS|EXT_ARG_NUM_THREADS=1|

次のセルで`EXTTOOL_PARAMS`に上記の環境変数を指定するステートメントを設定して下さい。

In [ ]:
# (例)
# EXTTOOL_PARAMS="
# EXT_ARG_OUTPUT_COUNT=1
# EXT_ARG_NUM_SAMPLES=300
# EXT_ARG_PAYLOAD_SIZE=1024
# EXT_ARG_NUM_THREADS=1
# "

EXTTOOL_PARAMS="
EXT_ARG_OUTPUT_COUNT=1
EXT_ARG_NUM_SAMPLES=300
EXT_ARG_PAYLOAD_SIZE=1024
EXT_ARG_NUM_THREADS=1
"

### perftool計測対象

perftoolコマンドの計測対象となるブローカに関するパラメータの指定を行います。

計測対象となるブローカのホスト名（またはIPアドレス）を指定して下さい。

In [ ]:
# (例)
# target_host=broker.example.org
# target_host=192.168.10.100

target_host=

ブローカのポート番号を指定して下さい。

In [ ]:
# (例)
# target_port=1883
# target_port=9092

target_port=1883

計測対象のトピック名を指定して下さい。

In [ ]:
# (例)
# target_topic=perftool-sinetstream-target

target_topic=perftool-sinetstream-target

ブローカの種別を指定して下さい。

In [ ]:
# (例)
# target_type=mqtt
# target_type=kafka

target_type=mqtt

ブローカにメッセージを送信する際の`consistency`を指定して下さい。

In [ ]:
# (例)
# target_type=AT_LEAST_ONCE
# target_type=AT_MOST_ONCE

target_consistency=AT_LEAST_ONCE

計測対象のブローカとの通信にて、上記で指定した以外のパラメータを指定する場合は`TARGET_PARAMS`に環境変数を指定するステートメントを設定して下さい。SINETStream設定ファイルのパラメータと環境変数との変換ルールを以下に示します。

* パラメータ名を全て大文字に変換する
* プレフィックスに `PERF_TGT_` をつける
* 複数階層をもつパラメータを指定する場合は階層の区切りに２文字のアンダースコア `__` を指定する

例えばパラメータ`data_compression`を指定する場合は環境変数`PERF_TGT_DATA_COMPRESSION`に値を設定します。

In [ ]:
# (例)
# TARGET_PARAMS="
# PERF_TGT_TLS=true
# PERF_TGT_DATA_COMPRESSION=true
# "

TARGET_PARAMS="
"

### perftool結果送信先

perftoolコマンドの計測結果を送信するKafkaブローカに関するパラメータの指定を行います。

Kafkaの外部公開ホスト名（またはIPアドレス）を指定してください。

In [ ]:
# (例)
# result_host=kafka.example.org
# result_host=192.168.10.100

result_host=

ブローカのポート番号を指定して下さい。

In [ ]:
# (例)
# result_port=9092

result_port=9092

送信先となるトピック名を指定して下さい。

In [ ]:
# (例)
# result_topic=perftool-sinetstream-target

result_topic=perftool-sinetstream-result

### perftool識別名

perftoolコマンドの実行環境を識別するための名前を次のセルで指定します。ここで指定した値は、計測結果を可視化する際に表示対象を選択するために必要となります。

In [ ]:
# (例)
# PERF_NAME=$target_topic
# PERF_NAME=$(hostname -f)
# PERF_NAME=perftool-1

PERF_NAME=$target_topic

## 資材の配置

環境を構築するために必要となる資材を配置します。

資材を配置するディレクトリを作成します。

In [ ]:
mkdir -p $target_dir

コンテナの構成を記述した`docker-compose.yml`を配置します。

In [ ]:
cp ../Perftool/docker/docker-compose-picamera.yml $target_dir/docker-compose.yml

配置した`docker-compose.yml`の内容を表示します。

In [ ]:
cat $target_dir/docker-compose.yml

perftoolコンテナの環境変数を設定する`.env.perftool`ファイルを作成します。

In [ ]:
cat > $target_dir/.env.perftool <<EOF
PERF_TGT_BROKERS=${target_host}:${target_port:-1883}
PERF_TGT_TOPIC=${target_topic}
PERF_TGT_TYPE=${target_type:-mqtt}
PERF_TGT_CONSISTENCY=${target_consistency:-AT_LEAST_ONCE}
NAME=${PERF_NAME:-$target_topic}
${TARGET_PARAMS:-}
EOF

`.env.perftool`ファイルの内容を確認します。

In [ ]:
cat $target_dir/.env.perftool

picameraコンテナの環境変数を設定する`.env.picamera`ファイルを作成します。

In [ ]:
cat > $target_dir/.env.picamera <<EOF
${PICAMERA2_PARAMS:-}
PICAMERA2_SIZE=${PICAMERA2_SIZE:?Image size must be set.}
SS_BROKERS=${kafka_hostname:?The hostname of Kafka must be set.}:${kafka_port:-9092}
SS_TOPIC=${picamera_topic:?The topic name must be set.}
SS_CONSISTENCY=AT_LEAST_ONCE
SCHEDULE=$SCHEDULE
${EXTTOOL_PARAMS:-}
EXT_SS_BROKERS=${result_host}:${result_port:-9092}
EXT_SS_TOPIC=${result_topic}
EXT_SS_TYPE=kafka
EXT_SS_CONSISTENCY=AT_LEAST_ONCE
EOF

`.env.picamera`ファイルの内容を確認します。

In [ ]:
cat $target_dir/.env.picamera

## コンテナの起動

利用するコンテナイメージを取得します。

In [ ]:
docker compose --project-directory ${target_dir} pull -q

コンテナを起動します。notebook環境で`docker compose up`を実行すると処理中の表示が煩雑なため、次のセルでは全ての出力結果を破棄しています。エラーや警告表示を確認する必要がある場合はnotebook環境ではなく、別窓でターミナルなどを開いて`docker compose up`コマンドを実行して下さい。

In [ ]:
docker compose --project-directory ${target_dir} up -d --remove-orphans >& /dev/null

コンテナの実行状況を確認します。コンテナのSTATUSがUpとなっていることを確認して下さい。

In [ ]:
docker compose --project-directory ${target_dir} ps